<a href="https://colab.research.google.com/github/jasreman8/LLMs-For-RAGs-II/blob/main/Foundational_RAG_Retrieval_Enhancement_using_Query_Expansion_and_Hypothetical_Questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning Objectives

- Illustrate how to improve the retrieval process of RAG using:
    - Query Expansion
    - Hypothetical Questions


# Setup

!pip install -q openai==1.66.3 \
                tiktoken==0.9.0 \
                langchain==0.3.20 \
                langchain-chroma==0.2.2 \
                langchain-openai==0.3.9 \
                chromadb==0.6.3

In [2]:
import os
import chromadb

from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

from google.colab import userdata

# Vector database is a specialized storage system designed to handle embeddings, dense numerical ...
# ... representations of data (such as text, images, or audio) that enable  fast similarity search.

In [3]:
openai_api_key = userdata.get('my_api_key')

# This LLM is going to help us do the text generation part in addition to allowing us to expand the queries.
# Seting low temperature is to make it more focused and deterministic.
llm = ChatOpenAI(
    api_key=openai_api_key,
    model='gpt-4o-mini',
    temperature=0.4
)

# Embedding modles help us convert text models into numerical vectors.
embedding_model = OpenAIEmbeddings(
    api_key=openai_api_key,
    model='text-embedding-3-small'
)

To illustrate the techniques of improving retrieval, let us set up an ephemeral Chroma database with a few documents.

In [4]:
chromadb_client = chromadb.EphemeralClient()

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


In [5]:
documents = [
    Document(
        id=1,
        page_content="We design, develop, manufacture, sell and lease high-performance fully electric vehicles and energy generation and storage systems, and offer services related to our products. We generally sell our products directly to customers, and continue to grow our customer-facing infrastructure through a global network of vehicle showrooms and service centers, Mobile Service, body shops, Supercharger stations and Destination Chargers to accelerate the widespread adoption of our products. We emphasize performance, attractive styling and the safety of our users and workforce in the design and manufacture of our products and are continuing to develop full self-driving technology for improved safety. We also strive to lower the cost of ownership for our customers through continuous efforts to reduce manufacturing costs and by offering financial and other services tailored to our products.",
        metadata={"year": 2023, "section": "business"}
    ),
    Document(
        id=2,
        page_content="We have previously experienced and may in the future experience launch and production ramp delays for new products and features. For example, we encountered unanticipated supplier issues that led to delays during the initial ramp of our first Model X and experienced challenges with a supplier and with ramping full automation for certain of our initial Model 3 manufacturing processes. In addition, we may introduce in the future new or unique manufacturing processes and design features for our products. As we expand our vehicle offerings and global footprint, there is no guarantee that we will be able to successfully and timely introduce and scale such processes or features.",
        metadata={"year": 2023, "section": "risk_factors"}
    ),
    Document(
        id=3,
        page_content="We recognize the importance of assessing, identifying, and managing material risks associated with cybersecurity threats, as such term is defined in Item 106(a) of Regulation S-K. These risks include, among other things: operational risks, intellectual property theft, fraud, extortion, harm to employees or customers and violation of data privacy or security laws. Identifying and assessing cybersecurity risk is integrated into our overall risk management systems and processes. Cybersecurity risks related to our business, technical operations, privacy and compliance issues are identified and addressed through a multi-faceted approach including third party assessments, internal IT Audit, IT security, governance, risk and compliance reviews. To defend, detect and respond to cybersecurity incidents, we, among other things: conduct proactive privacy and cybersecurity reviews of systems and applications, audit applicable data policies, perform penetration testing using external third-party tools and techniques to test security controls, operate a bug bounty program to encourage proactive vulnerability reporting, conduct employee training, monitor emerging laws and regulations related to data protection and information security (including our consumer products) and implement appropriate changes.",
        metadata={"year": 2023, "section": "cyber_security"}
    ),
    Document(
        id=4,
        page_content="The automotive segment includes the design, development, manufacturing, sales and leasing of high-performance fully electric vehicles as well as sales of automotive regulatory credits. Additionally, the automotive segment also includes services and other, which includes non-warranty after- sales vehicle services and parts, sales of used vehicles, retail merchandise, paid Supercharging and vehicle insurance revenue. The energy generation and storage segment includes the design, manufacture, installation, sales and leasing of solar energy generation and energy storage products and related services and sales of solar energy systems incentives.",
        metadata={"year": 2022, "section": "business"}
    ),
    Document(
        id=5,
        page_content="Since the first quarter of 2020, there has been a worldwide impact from the COVID-19 pandemic. Government regulations and shifting social behaviors have, at times, limited or closed non-essential transportation, government functions, business activities and person-to-person interactions. Global trade conditions and consumer trends that originated during the pandemic continue to persist and may also have long-lasting adverse impact on us and our industries independently of the progress of the pandemic.",
        metadata={"year": 2022, "section": "risk_factors"}
    ),
    Document(
        id=6,
        page_content="The German Umweltbundesamt issued our subsidiary in Germany a notice and fine in the amount of 12 million euro alleging its non-compliance under applicable laws relating to market participation notifications and take-back obligations with respect to end-of-life battery products required thereunder. In response to Tesla’s objection, the German Umweltbundesamt issued Tesla a revised fine notice dated April 29, 2021 in which it reduced the original fine amount to 1.45 million euro. This is primarily relating to administrative requirements, but Tesla has continued to take back battery packs, and filed a new objection in June 2021. A hearing took place on November 24, 2022, and the parties reached a settlement which resulted in a further reduction of the fine to 600,000 euro. Both parties have waived their right to appeal.",
        metadata={"year": 2022, "section": "legal_proceedings"}
    )
]

We can now point our vector store to the Chroma client and add these documents to the vector store as in previous modules.

In [6]:
# Setting up our vectorstore using Chroma client.
vectorstore = Chroma(
    collection_name="full_document_chunks",
    collection_metadata={"hnsw:space": "cosine"},
    embedding_function=embedding_model,
    client=chromadb_client
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [7]:
chromadb_client.list_collections()

['full_document_chunks']

In [8]:
vectorstore.add_documents(
    documents=documents
)

['1', '2', '3', '4', '5', '6']

In [9]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        'k': 3
    }
)

# Query Expansion

In query expansion, we ask the LLM to generate variations of the original user query. Then, we use each of these variations to retrieve the relevant context. The final context used is the unique set of documents that are retreived across all the query expansions.

In [10]:
query_expansion_system_message = """
You are an financial domain expert assisting in answering questions related to 10-k reports.
Perform query expansion on the question below. If there are multiple common ways of phrasing a user question \
or common synonyms for key words in the question, make sure to return multiple versions \
of the query with the different phrasings.

If there are acronyms or words you are not familiar with, do not try to rephrase them.

Return at least 3 versions of the question as a list.
Generate only a list of questions, each question in a new line.
Do not number the list of questions or use bullet points.
Do not mention anything before or after the list.
"""

# Over here, a template for a user message is created that will contain the original question.
# This template uses XML style tags to clearly delinate the question for the language model.
user_message_template="""
<Question>
{question}
</Question>
"""

In [11]:
user_input = "Any specific fines levied on the company in 2022?"

query_expansions = llm.invoke(
    [
        ('system', query_expansion_system_message),
        ('user', user_message_template.format(question=user_input))
    ]
)

query_expansions_list = query_expansions.content.strip().split("\n")
query_expansions_list

['Any particular penalties imposed on the company in 2022?  ',
 'Were there any fines charged against the company in 2022?  ',
 'Did the company face any specific fines in 2022?']

In [12]:
expanded_context_list = []

for query in query_expansions_list:
    expanded_context_list.extend([d.page_content for d in retriever.invoke(query)])

final_context_documents = set(expanded_context_list)
final_context_documents

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


{'Since the first quarter of 2020, there has been a worldwide impact from the COVID-19 pandemic. Government regulations and shifting social behaviors have, at times, limited or closed non-essential transportation, government functions, business activities and person-to-person interactions. Global trade conditions and consumer trends that originated during the pandemic continue to persist and may also have long-lasting adverse impact on us and our industries independently of the progress of the pandemic.',
 'The German Umweltbundesamt issued our subsidiary in Germany a notice and fine in the amount of 12 million euro alleging its non-compliance under applicable laws relating to market participation notifications and take-back obligations with respect to end-of-life battery products required thereunder. In response to Tesla’s objection, the German Umweltbundesamt issued Tesla a revised fine notice dated April 29, 2021 in which it reduced the original fine amount to 1.45 million euro. Thi

# Hypothetical Questions

In this approach, we generate 3 hypothetical questions that can be answered with each document chunk. These hypothetical questions are then seperately indexed into the vector database (along with the parent document chunk ids as metadata). For each query, we then retrieve relevant hypothetical questions first and the then retrieve the associated chunks as the second step. Note that the retrieval is focused on the comparison between the user query and hypothetical questions.

In [13]:
hypothetical_questions_system_message = """
Generate a list of exactly 3 hypothetical questions that the document presented in the input could be used to answer.
Generate only a list of questions, each question in a new line.
Do not number the questions or use bullet points.
Do not mention anything before or after the list.
"""

user_message_template = """
<Document>
{document}
</Document>
"""

In [14]:
hypothetical_questions = []

In [15]:
for document in documents:

    try:
        response = llm.invoke(
            [
                ('system', hypothetical_questions_system_message),
                ('user', user_message_template.format(document=document.page_content))
            ]
        )

        questions = response.content.strip()
    except Exception as e:
        questions = ""

    questions_list = questions.split("\n")

    for question in questions_list:

        questions_metadata = {
            'parent_chunk_id': document.id,
            'parent_collection': 'full_document_chunks'
        }

        hypothetical_questions.append(
            Document(
                page_content=question,
                metadata=questions_metadata
            )
        )

Let us look at the first set of hypothetical questions.

In [16]:
hypothetical_questions[0], hypothetical_questions[1], hypothetical_questions[2]

(Document(metadata={'parent_chunk_id': '1', 'parent_collection': 'full_document_chunks'}, page_content='What types of products does the company design and manufacture?  '),
 Document(metadata={'parent_chunk_id': '1', 'parent_collection': 'full_document_chunks'}, page_content='How does the company aim to enhance customer experience and product adoption?  '),
 Document(metadata={'parent_chunk_id': '1', 'parent_collection': 'full_document_chunks'}, page_content='What strategies does the company employ to reduce the cost of ownership for customers?'))

We can now index these hypothetical questions into a new collection.

In [17]:
hypothetical_questions_vectorstore = Chroma(
    collection_name="hypothetical_questions",
    collection_metadata={"hnsw:space": "cosine"},
    embedding_function=embedding_model,
    client=chromadb_client
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


 - Parallel indexing refers to the process of indexing large datasets or documents across multiple processors or nodes simultaneously to speed up the creation of the index for fast retrieval.

In [18]:
chromadb_client.list_collections()

['full_document_chunks', 'hypothetical_questions']

In [19]:
hypothetical_questions_vectorstore.add_documents(
    documents=hypothetical_questions
)

['4e839c91-aac4-4c8a-a2bc-3334b3e4d773',
 'c196fb2c-588e-4bd2-baf9-1cc14556af65',
 '8e713663-4051-4183-a0e3-6d587b865808',
 '6083e3ca-2c55-49f9-9ca4-125bed1618af',
 '1b4d6840-f66d-4c72-b91b-8fb0245fe3cb',
 'c63344a9-4baf-4c63-ba7d-f3a2325c6c2b',
 '48572e21-6383-4508-846d-b7c7c819b962',
 'c9d5f69e-c7e9-4523-9923-b22879de9cca',
 'b266b288-b7ea-461e-95e0-908debb13698',
 '17ed9118-4b70-4279-bfcd-09dad2d9f848',
 '58e02ba4-5592-4143-8289-271a32022b02',
 '2f9fd669-43f6-4778-b90a-7ab702fbedde',
 '305e22f7-c924-45f2-a53f-ecd84d740e2f',
 'c3a2ba4c-5b70-401a-af76-91084ac3f3e0',
 'b8f44ac4-d253-49b3-b553-7555b5ea0bba',
 '88a0d021-75f4-4445-86df-0b6f9964c094',
 '76160dfa-1f2f-4e69-9671-fab0f36b9570',
 '2e4fdfab-9e7b-4329-aaa7-0afd08c8c18b']

In [20]:
retriever = hypothetical_questions_vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 5}
)

In [21]:
user_query = "Any specific fines levied on the company in 2022?"

In [22]:
hypothetical_questions_retrieved = retriever.invoke(user_query)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


In [23]:
hypothetical_questions_retrieved

[Document(id='76160dfa-1f2f-4e69-9671-fab0f36b9570', metadata={'parent_chunk_id': '6', 'parent_collection': 'full_document_chunks'}, page_content='How much was the fine reduced to after the settlement reached on November 24, 2022?  '),
 Document(id='88a0d021-75f4-4445-86df-0b6f9964c094', metadata={'parent_chunk_id': '6', 'parent_collection': 'full_document_chunks'}, page_content='What was the original fine amount issued to Tesla by the German Umweltbundesamt for non-compliance?  '),
 Document(id='2e4fdfab-9e7b-4329-aaa7-0afd08c8c18b', metadata={'parent_chunk_id': '6', 'parent_collection': 'full_document_chunks'}, page_content='What actions did Tesla take in response to the initial fine notice from the German Umweltbundesamt?'),
 Document(id='c63344a9-4baf-4c63-ba7d-f3a2325c6c2b', metadata={'parent_chunk_id': '2', 'parent_collection': 'full_document_chunks'}, page_content='What challenges could arise as the company expands its vehicle offerings and global presence?'),
 Document(id='6083

In [24]:
vectorstore.get(
    ids=list(set([d.metadata['parent_chunk_id'] for d in hypothetical_questions_retrieved]))
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given


{'ids': ['2', '6'],
 'embeddings': None,
 'documents': ['We have previously experienced and may in the future experience launch and production ramp delays for new products and features. For example, we encountered unanticipated supplier issues that led to delays during the initial ramp of our first Model X and experienced challenges with a supplier and with ramping full automation for certain of our initial Model 3 manufacturing processes. In addition, we may introduce in the future new or unique manufacturing processes and design features for our products. As we expand our vehicle offerings and global footprint, there is no guarantee that we will be able to successfully and timely introduce and scale such processes or features.',
  'The German Umweltbundesamt issued our subsidiary in Germany a notice and fine in the amount of 12 million euro alleging its non-compliance under applicable laws relating to market participation notifications and take-back obligations with respect to end-of